In [4]:
import requests
from datetime import datetime, timedelta, UTC
import pandas as pd
from io import BytesIO
import json
from collections import defaultdict

session = requests.Session()
url = "http://ionbeam-ichange.ecmwf-ichange.f.ewcloud.host/api/v1/"
# url = "http://136.156.130.47/api/v1/"
url = "http://localhost:50310/api/v1/"
# url = "http://localhost:5002/api/v1/"

def api_get(path, *args, **kwargs):
    r = session.get(url + path, stream=True, *args, **kwargs)
    if not r.ok:
        print(f"API Error")
        print(json.dumps(r.json(), indent = 4))
    return r

def sort_by_platform(stations) -> dict[str, list[dict]]:
    by_platform = defaultdict(list)
    for s in stations:
        by_platform[s["platform"]].append(s)
    return by_platform

In [5]:
# This can take a long time...
all_stations = api_get("stations", params = {
}).json()

In [6]:
print("All stations")
all_by_platform = sort_by_platform(all_stations)
print({k : len(v) for k, v in all_by_platform.items()})
print()

All stations
{'meteotracker': 18711, 'smart_citizen_kit': 4}



## Total Meteotracker datasets

In [7]:
dates = [datetime.fromisoformat(s['time_span']["end"]) for s in all_by_platform['meteotracker']]
earliest, latest = min(dates), max(dates)
print(f"Meteotracker: {len(all_by_platform['meteotracker'])} tracks.")
print(f"Earliest ingested data: {earliest}")
print(f"Most recent ingested data: {latest}")

Meteotracker: 18711 tracks.
Earliest ingested data: 2023-01-20 12:23:39.075000+00:00
Most recent ingested data: 2025-01-21 11:39:59+00:00


## Total Acronet datasets

In [8]:
def get_all_granules_by_platform(platform):
    granules = api_get("list", params = {'platform': platform}).json()
    dates = [datetime.fromisoformat(d["datetime"]) for d in granules]
    # print(set(g["mars_request"]["platform"] for g in granules))
    assert all(g["mars_request"]["platform"] == platform for g in granules)
    return granules, dates

acronet_granules, dates = get_all_granules_by_platform(platform = "acronet")
if dates:
    earliest, latest = min(dates), max(dates)
    
    print(f"Acronet: {len(all_by_platform['acronet'])} distinct stations.")
    print(f"Earliest ingested data: {earliest.date()} Most recent ingested data: {latest.date()}")
    print(f"Ingested data: {len(acronet_granules)} acronet data granules.")

## Total Smart Citizen Kit Datasets

In [9]:
sck_granules, dates = get_all_granules_by_platform(platform = "smart_citizen_kit")
earliest, latest = min(dates), max(dates)

print(f"Smart Citizen Kit: {len(all_by_platform['smart_citizen_kit'])} distinct stations.")
print(f"Earliest ingested data: {earliest} Most recent ingested data: {latest}")
print(f"Ingested data: {len(sck_granules)} data granules.")

Smart Citizen Kit: 4 distinct stations.
Earliest ingested data: 2025-01-05 17:00:00+00:00 Most recent ingested data: 2025-01-21 12:00:00+00:00
Ingested data: 317 data granules.
